In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

class AdaptiveKalmanFilter:
    def __init__(self, beta_initial, P_initial, Q_initial, R_initial, adapt_rate=0.01):
        self.beta = beta_initial
        self.P = P_initial
        self.Q = Q_initial
        self.R = R_initial
        self.adapt_rate = adapt_rate

    def update(self, y):
        # Prediction
        beta_predicted = self.beta
        P_predicted = self.P + self.Q

        # Innovation
        innovation = y - beta_predicted
        innovation_variance = P_predicted + self.R

        # Update Q and R adaptively based on innovation
        self.Q = (1 - self.adapt_rate) * self.Q + self.adapt_rate * (innovation ** 2)
        self.R = (1 - self.adapt_rate) * self.R + self.adapt_rate * innovation_variance

        # Kalman Gain
        K = P_predicted / innovation_variance
        self.beta = beta_predicted + K * innovation
        self.P = (1 - K) * P_predicted

        return self.beta

# Load and prepare NASDAQ data for both 2008 and 2024
nasdaq_data = pd.read_csv("NasdaqData.csv")
nasdaq_data['Close'] = nasdaq_data['Close'].str.replace(',', '').astype(float)
nasdaq_data['Date'] = pd.to_datetime(nasdaq_data['Date'])
nasdaq_data_2008 = nasdaq_data[nasdaq_data['Date'].dt.year == 2008]

nasdaq_data_2024 = pd.read_csv("/Users/dobromiriliev/Documents/GitHub/MonteCarlo2008Simulation/Nasdaq2024.csv")
nasdaq_data_2024['Close'] = nasdaq_data_2024['Close'].str.replace(',', '').astype(float)
nasdaq_data_2024['Date'] = pd.to_datetime(nasdaq_data_2024['Date'])

# Initialize Kalman Filter for 2008 data
beta_initial_2008 = nasdaq_data_2008['Close'].iloc[0]
kalman_filter_2008 = AdaptiveKalmanFilter(beta_initial_2008, 1.0, 0.0001, 0.001, 0.02)
estimated_betas_2008 = [beta_initial_2008]
for price in nasdaq_data_2008['Close'][1:]:
    estimated_beta = kalman_filter_2008.update(price)
    estimated_betas_2008.append(estimated_beta)

# Plot 2008 Kalman Filter results
plt.figure(figsize=(14, 7))
plt.plot(nasdaq_data_2008['Date'], nasdaq_data_2008['Close'], label='Actual NASDAQ Close 2008', color='green')
plt.plot(nasdaq_data_2008['Date'], estimated_betas_2008, label='Estimated by Kalman Filter 2008', color='red')
plt.xlabel('Date')
plt.ylabel('NASDAQ Closing Value')
plt.title('Actual vs. Kalman Filter Estimated NASDAQ Closing Values (2008)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Initialize Kalman Filter for 2024 data
beta_initial_2024 = nasdaq_data_2024['Close'].iloc[0]
kalman_filter_2024 = AdaptiveKalmanFilter(beta_initial_2024, 1.0, 0.0001, 0.001, 0.02)
estimated_betas_2024 = [beta_initial_2024]
for price in nasdaq_data_2024['Close'][1:]:
    estimated_beta = kalman_filter_2024.update(price)
    estimated_betas_2024.append(estimated_beta)

# Plot 2024 Kalman Filter results
plt.figure(figsize=(14, 7))
plt.plot(nasdaq_data_2024['Date'], nasdaq_data_2024['Close'], label='Actual NASDAQ Close 2024', color='blue')
plt.plot(nasdaq_data_2024['Date'], estimated_betas_2024, label='Estimated by Kalman Filter 2024', color='orange')
plt.xlabel('Date')
plt.ylabel('NASDAQ Closing Value')
plt.title('Actual vs. Kalman Filter Estimated NASDAQ Closing Values (2024)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Simulate AR(1) process for 2008 data
phi = 0.9
sigma = 0.5
nasdaq_dates = nasdaq_data_2008['Date']
X_ar = [beta_initial_2008]
for _ in range(1, len(nasdaq_dates)):
    X_ar.append(phi * X_ar[-1] + np.random.normal(0, sigma))

# Plot AR(1) process for 2008
plt.figure(figsize=(14, 7))
plt.plot(nasdaq_dates, X_ar, label='AR(1) Process 2008', color='orange')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('AR(1) Process Simulation for NASDAQ 2008 Data')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(10))  # Set maximum number of x-axis labels to display
plt.tight_layout()
plt.show()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa6 in position 10: invalid start byte